In [ ]:
import json
import uproot
import uproot_methods
import awkward
import numpy as np
from functools import partial
%matplotlib nbagg

import cloudpickle, lz4.frame as lz4f

import fnal_column_analysis_tools.processor as processor
import fnal_column_analysis_tools.hist as hist
from fnal_column_analysis_tools.hist import plot
import matplotlib.pyplot as plt

In [ ]:
with open('files.json') as fin:
    samples = json.load(fin)

samples['GluGluHToBB_M125_LHEHpT_250-Inf_13TeV_amcatnloFXFX_pythia8'] = samples['GluGluHToBB_M125_LHEHpT_250-Inf_13TeV_amcatnloFXFX_pythia8'][:2]
print("\n".join(samples.keys()))

In [ ]:
class HiggsGenComparison(processor.ProcessorABC):
    def __init__(self):
        dataset_axis = hist.Cat("dataset", "Primary dataset")
        pt_axis = hist.Bin("pt", r"$p_{T,h}$ [GeV]", 120, 0., 1200.)
        widx_axis = hist.Bin("widx", "Weight index", 9, 0, 9)
        
        self._accumulator = processor.dict_accumulator({
            'hpt': hist.Hist("Counts", dataset_axis, pt_axis, widx_axis),
        })
    
    @property
    def accumulator(self):
        return self._accumulator
    
    def nanoObject(self, df, prefix):
        branches = set(k.decode('ascii') for k in df.available if k.decode('ascii').startswith(prefix))
        p4branches = [prefix + k for k in ['pt', 'eta', 'phi', 'mass']]
        branches -= set(p4branches)
        objp4 = uproot_methods.TLorentzVectorArray.from_ptetaphim(*[df[b] for b in p4branches])
        branches = {k[len(prefix):]: df[k] for k in branches}
        obj = awkward.JaggedArray.zip(p4=objp4, **branches)
        return obj

    def process(self, df):
        output = self.accumulator.identity()
        dataset = df['dataset']
        
        genp = self.nanoObject(df, 'GenPart_')
        hidx = (genp['pdgId']==25) & (genp['statusFlags']&(1<<7)).astype(bool)
        higgs = genp[hidx]
        
        if dataset == 'GluGluHToBB_M-125_13TeV_powheg_MINLO_NNLOPS_pythia8':
            output['hpt'].fill(dataset=dataset,
                               pt=higgs['p4'].pt.flatten(),
                               widx=4,
                               weight=df['genWeight'])
        else:
            scalew = df['LHEScaleWeight'].regular()
            for iw in range(9):
                output['hpt'].fill(dataset=dataset,
                                   pt=higgs['p4'].pt.flatten(),
                                   widx=iw,
                                   weight=df['genWeight']*scalew[:,iw])
        
        return output

    def postprocess(self, accumulator):
        return accumulator
    

class GetSumW(processor.ProcessorABC):
    def __init__(self):

        self._accumulator = processor.dict_accumulator({
            'sumw': processor.defaultdict_accumulator(float),
            'scaleSumw': processor.defaultdict_accumulator(partial(np.zeros, 9)),
        })
    
    @property
    def accumulator(self):
        return self._accumulator

    def process(self, df):
        output = self.accumulator.identity()
        dataset = df['dataset']
        
        output['sumw'][dataset] += np.sum(df['genEventSumw'])
        if dataset == 'GluGluHToBB_M-125_13TeV_powheg_MINLO_NNLOPS_pythia8':
            scalew = np.ones(shape=(1,9))*df['genEventSumw']
        else:
            scalew = df['LHEScaleSumw'].regular()
        output['scaleSumw'][dataset] += np.sum(scalew*df['genEventSumw']/scalew[:,4], axis=0)
        return output

    def postprocess(self, accumulator):
        return accumulator

In [ ]:
output = processor.run_uproot_job(samples,
                                  treename='Events',
                                  processor_instance=HiggsGenComparison(),
                                  executor=processor.futures_executor,
                                  executor_args={'workers': 8},
                                  chunksize=500000,
                                 )

sumw = processor.run_uproot_job(samples,
                                  treename='Runs',
                                  processor_instance=GetSumW(),
                                  executor=processor.futures_executor,
                                  executor_args={'workers': 8},
                                  chunksize=500000,
                                 )

output += sumw

In [ ]:
with lz4f.open("genHpt.cpkl.gz", "w") as fout:
    cloudpickle.dump(output, fout)

In [ ]:
with lz4f.open("genHpt.cpkl.gz") as fin:
    output = cloudpickle.load(fin)

In [ ]:
lumi = 1000.
xs = {
    # nominal 48.85, BR 0.5824
    'GluGluHToBB_M125_13TeV_powheg_pythia8': 21.46, # HIG-RunIIFall18wmLHEGS-00565
    'GluGluHToBB_M125_13TeV_amcatnloFXFX_pythia8': 33.30,
    'GluGluHToBB_M125_LHEHpT_250-Inf_13TeV_amcatnloFXFX_pythia8': 0.3447, # HIG-RunIIFall18wmLHEGS-00567
    'GluGluHToBB_M-125_13TeV_powheg_MINLO_NNLOPS_pythia8': 27.80,
    'VBFHToBB_M-125_13TeV_powheg_pythia8_weightfix': 3.782, # HIG-RunIIFall18wmLHEGS-00568
}
scale = {k: lumi*xs[k]/v for k,v in output['sumw'].items()}
hpt = output['hpt'].copy()
hpt.scale(scale, axis='dataset')

In [ ]:
fig, ax, _ = plot.plot1d(hpt.project('widx', 4), overlay='dataset')
ax.set_xlim(250, None)
ax.set_ylim(.01, None)
ax.set_yscale('log')

In [ ]:
edges = hpt.axis('pt').edges()
values = {k[0]: v for k,v in hpt.values(sumw2=True, overflow='over').items()}
for k in values:
    sumw = np.cumsum(values[k][0][::-1], axis=0)[::-1]
    sumw2 = np.cumsum(values[k][1][::-1], axis=0)[::-1]
    nom = sumw[:,4]
    statw = np.sqrt(sumw2)[:,4]
    valid = [0, 1, 3, 4, 5, 7, 8]
    scaleup = np.max(sumw[:,valid], axis=1)
    scaledn = np.min(sumw[:,valid], axis=1)
    values[k] = (nom, statw, scaleup, scaledn)

In [ ]:
# https://cds.cern.ch/record/2669113/files/LHCHXSWG-2019-002.pdf
# Table 8
edges_xswg = np.array([   400,   430,   450,  500,  550,  600,  650,  700,  750,  800])
values_xswg = np.array([32.03, 22.05, 17.37, 9.66, 5.54, 3.24, 1.94, 1.15, 0.69, 0.41])

from scipy.interpolate import interp1d
interp_xswg = interp1d(edges_xswg, values_xswg, kind='cubic')

In [ ]:
plt.rcParams.update({
    'font.size': 14,
    'axes.titlesize': 14,
    'axes.labelsize': 14,
    'xtick.labelsize': 12,
    'xtick.direction': 'in',
    'xtick.top': True,
    'ytick.labelsize': 12,
    'ytick.direction': 'in',
    'ytick.right': True,
    'legend.fontsize': 12,
})

In [ ]:
fig, (ax, rax) = plt.subplots(2, 1, figsize=(6,6), gridspec_kw={"height_ratios": (3, 1)}, sharex=True)
fig.subplots_adjust(hspace=.07)

def draw(key, label, scale=True):
    nom, statw, scaleup, scaledn = values[key]
    line, = ax.step(x=edges, y=nom, where='post', label=label)
    fill = ax.fill_between(x=edges, y1=nom - statw, y2=nom + statw,
                           facecolor='none', edgecolor=line.get_color(), hatch='///', linewidth=0, step='post', label='_')
    if scale:
        fills = ax.fill_between(x=edges, y1=scaledn, y2=scaleup,
                           color=line.get_color(), alpha=0.2, step='post', label='_')
    
    cut = (edges >= edges_xswg[0]) & (edges < edges_xswg[-1])
    rline, = rax.step(edges[cut], nom[cut]/interp_xswg(edges[cut]), color=line.get_color(), where='post')
    return line

l0, = ax.plot(edges_xswg, values_xswg, linestyle='-', color='k', linewidth=2, label='LHCHXSWG-2019-002')


# http://home.fnal.gov/~ncsmith/gg_H_quark-mass-effects_NNPDF31_13TeV_M125_slc6_amd64_gcc630_CMSSW_9_3_0.powheg.input
# https://arxiv.org/pdf/1111.2854.pdf
l1 = draw('GluGluHToBB_M125_13TeV_powheg_pythia8', 'POWHEG gg_H_quark-mass-effects')


l2 = draw('GluGluHToBB_M125_LHEHpT_250-Inf_13TeV_amcatnloFXFX_pythia8', r'MG5 $m_{t}=\infty$')
#lx = draw('GluGluHToBB_M125_13TeV_amcatnloFXFX_pythia8', r'MG5 $m_{t}=\infty$')


# https://github.com/cms-sw/genproductions/blob/b8c44435bd90b2e11cdd21ba596113b1b278a38c/bin/Powheg/production/2017/13TeV/Higgs/HJ_MiNLO_NNLOPS_NNPDF31_13TeV/HJ_MiNLO_NNLOPS_NNPDF31_13TeV.input
# https://arxiv.org/pdf/1212.4504.pdf
l3 = draw('GluGluHToBB_M-125_13TeV_powheg_MINLO_NNLOPS_pythia8', 'POWHEG MiNLO', scale=False)

ax.set_yscale('log')
ax.set_ylim(5e-2, 1e3)
ax.set_ylabel(r'$\sigma(gg\to h, p_{T,h}\geq p_{T,h}^{cut})$ [fb]')

from matplotlib.patches import Patch
statpatch = Patch(facecolor='none', edgecolor='k', hatch='///', linewidth=0, label='Stat. Unc.')
scalepatch = Patch(facecolor='black', alpha=0.2, label=r'$\mu_R,\mu_F$ Unc.')
litems = [l0, l1, l2, l3, statpatch, scalepatch]
ax.legend(litems, [x.get_label() for x in litems])


rax.set_xlim(250, 1200)
rax.set_ylim(0, 3)
rax.axhline(1, alpha=0.5, color='k', linestyle='--')
rax.set_ylabel('x / HXSWG')
rax.set_xlabel(r'$p_{T,h}^{cut}$ [GeV]')

In [ ]:
fig.savefig('hxswg_cmsgenerators_comparison.pdf')